Let's start by loading the necessary libraries and our data.

In [1]:
import numpy as np
import pandas as pd

In [2]:
game_data = pd.read_csv('../Data/games-data.csv')

Up next is a function that changes the column names:
>game -> name of the game  
>console -> name of the console in which the game was released  
>release_date -> date in which the game got released to the public  
>critic_score -> average score given by professional critics  
>user_score -> average score given by the users of the metacritic website  
>dev -> name of the developer studio that worked on the game  
>player_support -> amount of players that the game supports in a single session  
>num_critics -> number of critics that left a review on the game  
>num_users -> number of users that left a review on the game  
>genre -> list of genres that best represent the game (unchanged with the function)

In [3]:
%%writefile functions.py

import numpy as np
import pandas as pd 

def clean_col_names(df):
    """
        Takes our dataframe and changes the column names into ones that better represent the contents of that column
        Input -> df Dataframe needing column name cleaning
        Output -> df1 Dataframe with columns renamed
    """
    
    df1 = df.copy()
    df1.rename(columns = {'name' : 'game', 'platform' : 'console', 'r-date' : 'release_date',
                         'score' : 'critic_score', 'user score' : 'user_score',
                         'developer' : 'dev', 'players' : 'player_support',
                         'critics' : 'num_critics', 'users' : 'num_users'}, inplace = True )
    return df1

Overwriting functions.py


Next is a function to get rid of rows where the values of the numeric columns doesn't make too much sense.

In [4]:
%%writefile -a functions.py
def weird_rows(df):
    """
        Takes our dataframe and eliminates rows with values in the numerical columns that are simply no good
        Input -> df Dataframe needing row cleaning
        Output -> df1 Dataframe with no strange rows
    """
    
    df1 = df.copy()
    df1 = df1[ (df['user_score'] != 'tbd') & (df['num_users'] != 0) & (df['num_critics'] != 0) ]
    return df1

Appending to functions.py


Afterwards there is the need to fill some null values present in the "player_support" column, so here is funcion that does that.

In [5]:
%%writefile -a functions.py
def support_fill(df):
    """
        Takes our dataframe and fills the values in the "player_support" column with reasonable ones
        Input -> df Dataframe with a few null values in the "player_support" column
        Output -> df1 Dataframe cleaned of null values
    """
    
    df1 = df.copy()
    df1['player_support'].fillna('No info', inplace=True)
    return df1

Appending to functions.py


The values on columns "critic_score" and "user_score" are on a different scale (0-100 and 0-10 respectively), so there's the need to tranform one so it's on the same scale as the other. Since "user_score" also happens to be as object type will be simultaniously changing that to integer.

In [6]:
%%writefile -a functions.py
def fix_user_score(x):
    """
        Simply multiplies a number by 10 an returns the result as an integer
        Input -> any numeric value x, even if it's not cast as a numerical type
        Output -> produc y cast as an integer
    """
    
    y = float(x) * 10
    return int(y)

Appending to functions.py


To get some extra details out of the data, there is going to be a function which creates a new column named "score_diff".  
This new column is the result of the difference between user_score and critic_score, which due to input means that:
>a positive value means the user score is bigger than the critic score  
>a negative value means the critic score is bigger than the user score

In [7]:
%%writefile -a functions.py
def score_diff_col(df):
    """
        Takes our dataframe and adds a new column with the difference between the two score columns
        Input -> df Dataframe
        Output -> df1 Dataframe with the newly created column
    """
    
    df1 = df.copy()
    df1['score_diff'] = df['user_score']-df['critic_score']
    return df1

Appending to functions.py


The next step is to create a function that changes the "release_date" column into datetime type and extracts the month and year as integers into new columns aptly named "release_month" and "release_year" respectively.  
These new columns could potentially hold interesting information regarding when most games get released.

In [8]:
%%writefile -a functions.py
def date_fix(df):
    """
        Takes our dataframe, changes the date column to the correct type and adds 2 new columns
        Input -> df Dataframe
        Output -> df1 Dataframe with the 2 new columns
    """
    
    df1 = df.copy()
    df1['release_date'] = pd.to_datetime(df['release_date'])
    df1['release_year'] = pd.DatetimeIndex(df1['release_date']).year
    df1['release_month'] = pd.DatetimeIndex(df1['release_date']).month
    return df1

Appending to functions.py


It is now time to strat grouping the some of the values from our categorical columns, so the next few functions will be for that.

In [9]:
%%writefile -a functions.py
def console_groupping(row):
    """
        Takes a string an aggregates it into a group based on it's value
        Input -> any possible string
        Output -> string whose value depends on the input
    """
    
    if "PlayStation" in row or row == "PSP":
        return "PS"
    elif "Xbox" in row:
        return "Xbox"
    elif row == "Dreamcast":
        return "Sega"
    elif row == "Stadia":
        return "Google"
    elif row == "PC":
        return "PC"
    else:
        return "Nintendo"

Appending to functions.py


In [10]:
%%writefile -a functions.py
def fix_player_count(row):
    """
        Takes a string and aggregates it into a group based on it's value
        Input -> any possible string
        Output -> string whose value depends on the input
    """
    
    if row == "No info":
        return "No info"
    elif row == "1 Player" or row == "No Online Multiplayer":
        return "Singleplayer"
    elif row=="1-2 " or row=="1-3 " or row=="1-4 " or row=="2 " or row=="Up to 3 " or row=="Up to 4 " or row=="2  Online"  or row=="3  Online"  or row=="4  Online" :
        return "Co-op"
    else:
        return "Multiplayer"

Appending to functions.py


In [11]:
%%writefile -a functions.py
def genre_cleaning(row):
    """
        Takes a string with a comma, and returns the substring prior the first comma
        Input -> any possible string as long as it has a comma
        Output -> everything before the first comma
    """
    
    g=row.split(",")
    return g[0]

Appending to functions.py


In [12]:
%%writefile -a functions.py
def genre_cleaning_sec(row):
    """
        Takes a string with a comma, and returns the substring between the first and second commas
        Input -> any possible string as long as it has a comma
        Output -> everything between the first and second commas; if there is no second comma, returns "No secondary genre"
    """
    
    g=row.split(",")
    if len(g)>1:
        return g[1]
    else:
        return 'No secondary genre'

Appending to functions.py


In [13]:
%%writefile -a functions.py
def narrow_genres(df):
    """
        Takes our dataframe and agregates most of the main and secondary genres while retaining the 10 or so with highest count
        Input -> df Dataframe
        Output -> df1 Dataframe with aggregated main and secondary genres
    """
    
    df1 = df.copy()
    df1['main_genre'] = np.where( df['main_genre'].isin( ['Action', 'Action Adventure', 'Role-Playing', 'Sports', 'Strategy', 'Miscellaneous', 'Adventure', 'Driving', 'Simulation'] ) , df['main_genre'], 'Other')
    df1['sec_genre'] = np.where( df['sec_genre'].isin( ['General', 'Shooter', 'Platformer', 'Traditional', 'Racing', 'Action RPG', 'Real-Time', 'Miscellaneous', 'Fighting', 'Turn-Based', 'Beat-\'Em-Up'] ) , df['sec_genre'], 'Other')
    return df1

Appending to functions.py


In [14]:
%%writefile -a functions.py
def dev_mess(row):
    """
        Takes a string and aggregates it into a specific group
        Input -> any possible string
        Output -> string whose value depends on the input
    """
    
    if "Capcom" in row:
        return "Capcom"
    elif "TelltaleGames" in row:
        return "Telltale"
    elif row.startswith("EA") or "ElectronicArts" in row:
        return "EA"
    elif "Konami" in row:
        return "Konami"
    elif "Nintendo" in row:
        return "Nintendo"
    elif "Ubisoft" in row:
        return "Ubisoft"
    elif "Bandai" in row or "Namco" in row:
        return "BandaiNamco"
    elif "SquareEnix" in row or "SquareSoft" in row:
        return "SquareEnix"
    else:
        return "Other"

Appending to functions.py


Since we have games released over a vast amount of years, it would be fairly interesting to compute the yearly amount of user reviews a game gets. We could take this new column as theclosest thing to how popular a game really is.

In [15]:
%%writefile -a functions.py
def yearly_reviews(df):
    """
        Takes our dataframe and adds a new column containing the amount of user reviews that game gets per year
        Input -> df Dataframe
        Output -> df1 Dataframe with the new column
    """
    
    df1=df.copy()
    max_year = df1['release_year'].max()
    df1['yearly_num_reviews'] = ( ( df1['num_users']) / ( (max_year + 1) - df1['release_year'] ) )
    return df1

Appending to functions.py


And the final individual function will simply rearrange the column order into one that makes sense personally

In [16]:
%%writefile -a functions.py
def arrange_col(df):
    """
        Takes our dataframe and changes the order in which the columns appear, as well as sorting rows by year and montgh
        Input -> df Dataframe
        Output -> df1 Dataframe with the column order changed
    """
    
    df1 = df.copy()
    df1 = df1[['game', 'console', 'console_brand', 'dev', 'main_genre', 'sec_genre', 'player_support',
               'critic_score', 'user_score', 'score_diff','num_critics', 'num_users' , 
               'yearly_num_reviews', 'release_date', 'release_month', 'release_year']]
    df1.sort_values(by=["release_year",'release_month'],ascending=True, inplace=True)
    df1.reset_index(drop=True, inplace=True)
    return df1

Appending to functions.py


Now that we have all the necessary functions it's to make our mega function which executes all other functions and returns our fully cleaned data.

In [17]:
%%writefile -a functions.py
def clean_full_data(df):
    """
        Takes our dataframe and inputs all the necessary functions to fully clean it
        Input -> df Dataframe raw and yet untreated
        Output -> f_data Dataframe fully cleaned
    """
    
    df1 = df.copy()
    
    # start by changing column names
    col_change = clean_col_names(df1)
    
    # need to get rid of rows with nonsensical values
    fixed_rows = weird_rows(col_change)
    
    # time to fill the null values in the player_support column
    no_nulls = support_fill(fixed_rows)
    
    # apply the correct function that scales the values in user_score
    no_nulls['user_score'] = no_nulls['user_score'].apply(fix_user_score)
    
    # adding the new column
    score_diff_df = score_diff_col(no_nulls)
    
    # changing date column type and adding month and year columns
    data = date_fix(score_diff_df)
    
    # begin applying the functions that clean categorical data
    # first, is adding the console_brand column
    data['console_brand'] = data['console'].apply(console_groupping)
    
    # second is grouping the values from player_support so we don't have 50 unique values
    data['player_support'] = data['player_support'].apply(fix_player_count)
    
    # third is collecting the main and secondary genres
    data['main_genre']=data['genre'].apply(genre_cleaning)
    data['sec_genre']=data['genre'].apply(genre_cleaning_sec)
    
    # fourth is reducing the amount of each genre type by grouping most of the unique values
    data_narrow = narrow_genres(data)

    # fifth is cleaning up the dev column by group the vast majority of them while keeping the most common values
    data_narrow['dev'] = data_narrow['dev'].apply(dev_mess)
    
    # sixth is adding a new column which computes the amount of reviews per year a game gets
    data_yearly = yearly_reviews(data_narrow)
    
    # finally we change the order of columns and sort by release year and month
    f_data = arrange_col(data_yearly)
    
    return f_data

Appending to functions.py


In [18]:
import functions
# time to aplly the mega function (and pray)

final_data = functions.clean_full_data(game_data)

In [19]:
# this is to manually compare with the end of the exploratory notebook

display(final_data.head())
print(final_data.shape)

,game,console,console_brand,dev,main_genre,sec_genre,player_support,critic_score,user_score,score_diff,num_critics,num_users,yearly_num_reviews,release_date,release_month,release_year
0,Full Throttle,PC,PC,Other,Adventure,General,No info,86,86,0,8,181,6.961538,1995-04-30,4,1995
1,Duke Nukem 3D,PC,PC,Other,Action,Shooter,Co-op,89,87,-2,8,461,18.440000,1996-01-29,1,1996
2,Sid Meier's Civilization II,PC,PC,Other,Strategy,Turn-Based,Singleplayer,94,88,-6,7,450,18.000000,1996-02-29,2,1996
3,Bad Mojo,PC,PC,Other,Action Adventure,Other,Singleplayer,74,69,-5,10,13,0.520000,1996-02-29,2,1996
4,Resident Evil,PlayStation,PS,Capcom,Action Adventure,Other,Singleplayer,91,90,-1,8,526,21.040000,1996-03-30,3,1996


(16641, 16)


In [20]:
final_data.to_csv('../Data/cleaned_data.csv', index=False)